In [8]:
import open3d as o3d
import numpy as np

In [9]:
dbpath = r"./resampledPML/"
mesh_path = dbpath + r"AircraftBuoyant/m1342.obj"

mesh = o3d.io.read_triangle_mesh(mesh_path)
mesh.compute_vertex_normals()

pcd = mesh.sample_points_poisson_disk(750)
alpha = 0.3
print(f"alpha={alpha:.3f}")
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha)
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)

alpha=0.300
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: De ingang is ongeldig. 


In [10]:
import open3d as o3d
import numpy as np
import os
import glob
import pandas as pd
import pymeshlab as pml
import time
import matplotlib as plt
import copy


def o3d_to_pml(mesh):
    tmp_file_name = "temp.obj"
    o3d.io.write_triangle_mesh(tmp_file_name, mesh)
    # if not os.path.exists(tmp_file_name):
    #     print(f"File '{tmp_file_name}' doesn't exist. Waiting for 1 second...")
    #     time.sleep(1)    
    mesh_pml = pml.MeshSet()
    mesh_pml.load_new_mesh(tmp_file_name)
    os.remove(tmp_file_name)
    return mesh_pml

In [11]:
def consistent_normals(mesh_path):
    mesh = o3d.io.read_triangle_mesh(mesh_path)
    #o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)
    # render normals and make them consisitent
    pcd = mesh.sample_points_poisson_disk(7000)
    pcd.normals = o3d.utility.Vector3dVector(np.zeros((1, 3)))  # invalidate existing normals
    pcd.estimate_normals()
    pcd.orient_normals_consistent_tangent_plane(100)
    o3d.visualization.draw_geometries([pcd], point_show_normal=True)

    #convert pointcloud back to mesh
    #mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=4)
    
    for i in range(4,12):
        mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=i)[0]
        if mesh.is_watertight() == True:
            #o3d.visualization.draw_geometries([mesh], point_show_normal=True)
            #mesh = remove_smallest(mesh)
            return mesh
 


In [5]:
dbpath = r"./resampledPML/"
mesh_path = dbpath + r"AircraftBuoyant/m1340.obj"
mesh = consistent_normals(mesh_path)
print(mesh.is_watertight())
v = o3d.geometry.TriangleMesh.get_volume(mesh)
print(v)

True
0.028230478040506754


In [6]:
def remove_smallest(mesh):
    #o3d.visualization.draw_geometries([mesh])

    with o3d.utility.VerbosityContextManager(
            o3d.utility.VerbosityLevel.Debug) as cm:
        triangle_clusters, cluster_n_triangles, cluster_area = (
            mesh.cluster_connected_triangles())
    triangle_clusters = np.asarray(triangle_clusters)
    cluster_n_triangles = np.asarray(cluster_n_triangles)
    cluster_area = np.asarray(cluster_area)

    #remove smaller cluster
    mesh_0 = copy.deepcopy(mesh)
    triangles_to_remove = cluster_n_triangles[triangle_clusters] < 1000
    mesh_0.remove_triangles_by_mask(triangles_to_remove)
    #o3d.visualization.draw_geometries([mesh_0])
    return mesh_0

In [7]:
dbpath = r"./resampledPML/"
mesh_path = dbpath + r"AircraftBuoyant/m1340.obj"
mesh = consistent_normals(mesh_path)
v = o3d.geometry.TriangleMesh.get_volume(mesh)
print(v)
dbpath = r"./test_normals/"
new_path = dbpath + r"m1356.obj"
mesh = o3d_to_pml(mesh)
mesh.save_current_mesh(new_path)

0.028198002723665003


In [43]:
dbpath = r"./test_normals/"
mesh_path = dbpath + r"m1342.obj"
mesh = consistent_normals(mesh_path)

KeyboardInterrupt: 

***make alpha hull***

In [23]:
dbpath = r"./resampledPML/"
mesh_path = dbpath + r"AircraftBuoyant/m1339.obj"

def alpha_shape(mesh_path, alpha):
    mesh = o3d.io.read_triangle_mesh(mesh_path)
    pcd = gt_mesh.sample_points_poisson_disk(5000)
    mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha)
    return mesh


In [24]:
alpha = 0.2
dbpath = r"./resampledPML/"
mesh_path = dbpath + r"AircraftBuoyant/m1339.obj"

mesh = alpha_shape(mesh_path, 0.2)
o3d.visualization.draw_geometries([mesh], mesh_show_back_face=False)

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: De ingang is ongeldig. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: De aangevraagde overdrachtsbewerking wordt niet ondersteund. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: De ingang is ongeldig. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: De ingang is ongeldig. 


In [43]:
import trimesh
import scipy

In [64]:
def fix_mesh(mesh_path):
    mesh = o3d.io.read_triangle_mesh(mesh_path)
    #o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)
    # render normals and make them consisitent
    pcd = mesh.sample_points_poisson_disk(7000)
    pcd.normals = o3d.utility.Vector3dVector(np.zeros((1, 3)))  # invalidate existing normals
    pcd.estimate_normals()
    pcd.orient_normals_consistent_tangent_plane(100)
    #radii = [0.005, 0.01, 0.02, 0.04]
    radii = None


    nn_distances = pcd.compute_nearest_neighbor_distance()
    avg_nn_distance = np.mean(nn_distances)
    radius_of_ball = 3 * avg_nn_distance
    # heuristic to estimate the radius of a rolling ball
    #if radii is None:
    #    distances = pcd.compute_nearest_neighbor_distance()
    #    avg_dist = np.mean(distances)
    #    radius = 1.5 * avg_dist   
    #    radii = [radius, radius * 2]

    radii = [radius_of_ball, radius_of_ball]
    #radii = [0.03, 0.04, 0.05, 0.1]
    mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd, o3d.utility.DoubleVector(radii))
    o3d.visualization.draw_geometries([mesh])
    print(mesh)
    mesh = mesh.fill_holes() #hole_size: float = 1000000.0
    #mesh = trimesh.Trimesh(np.asarray(mesh.vertices), np.asarray(mesh.triangles), vertex_normals=np.asarray(mesh.vertex_normals))
    # try to fix normals with Trimesh
    #mesh.fix_normals()
    # save mesh:
    # mesh.export('../logs/mesh.obj')
    
    return mesh

In [65]:
dbpath = r"./resampledO3D/"
#mesh_path = dbpath + r"AircraftBuoyant/m1339.obj"
mesh_path = dbpath + r"Apartment/D00045.obj"
mesh = fix_mesh(mesh_path)
print(mesh.is_watertight())
#v = o3d.geometry.TriangleMesh.get_volume(mesh)
print(v)

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: De aangevraagde overdrachtsbewerking wordt niet ondersteund. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: De ingang is ongeldig. 
TriangleMesh with 7000 points and 9145 triangles.


AttributeError: 'open3d.cpu.pybind.geometry.TriangleMesh' object has no attribute 'fill_holes'

In [68]:
dbpath = r"./resampledO3D/"
mesh_path = dbpath + r"AircraftBuoyant/m1339.obj"
#mesh_path = dbpath + r"Apartment/D00045.obj"

mesh = o3d.io.read_triangle_mesh(mesh_path)
print(mesh)
print(mesh.is_watertight())
v = o3d.geometry.TriangleMesh.get_volume(mesh)
print(v)

TriangleMesh with 7363 points and 24692 triangles.
False


RuntimeError: [Open3D Error] (double __cdecl open3d::geometry::TriangleMesh::GetVolume(void) const) D:\a\Open3D\Open3D\cpp\open3d\geometry\TriangleMesh.cpp:1201: The mesh is not watertight, and the volume cannot be computed.
